In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from copy import deepcopy as dp

In [ ]:
#We use literal_eval to convert the inputted value into a dictionary in the format that we want it
from ast import literal_eval

In [ ]:
def convert_string_list_dic_into_actual(data):
    for i in range(len(data)):
        temp = data[i].strip('[').strip(']').replace("{", "").replace("}", "").split(',')
    
        convert_version = []
        for _ in range(int(len(temp) // 2)):
            convert_version.append(literal_eval("{" + f"{temp[0]}" + "," +f"{temp[1]}" + "}"))
            temp = temp[2:]
        data[i] = convert_version

In [ ]:
#Function for concatenating string of genres or languages together
def concate_string_list_dict_into_simple_string(data):
    for index in range(len(data)):
        temp_string = ""
        for element in data.iloc[index]:
            temp_string += f"{element['name']} "
        data.iloc[index] = temp_string

In [ ]:
#reading the file
movie_data = pd.read_csv('./the_movies_dataset/movies_metadata.csv')

In [ ]:
# Movie summary data, fill null value with empty string
movie_summary = movie_data['overview'].fillna('')

# Movie title data
movie_title_data = movie_data['original_title']

# Map movie title to its index
movie_to_index = pd.Series(movie_data.index, index=movie_data['title']).drop_duplicates()

I start investigating and wrangle the data for null or incorrect data types.

In [ ]:
movie_data.columns

I need: 
- genres : list[dict[]] (after conversion)
- popularity: float
- runtime: float
- spoken_languages : list[dict[]] -> after conversion
- vote_average: float

In [ ]:
movie_data.isnull().sum()

Convert movie_genres into list[dict[]] for easier use

In [ ]:
#Movie genre
movie_genres = movie_data['genres'].fillna('[{}]')

In [ ]:
#Movie language
movie_spoken_language = movie_data['spoken_languages'].fillna('[]')

In [ ]:
convert_string_list_dic_into_actual(movie_genres)
convert_string_list_dic_into_actual(movie_spoken_language)

In [ ]:
movie_spoken_language

Let's just form a concatenated string of genres together so that I can use TF-IDF for values.

In [ ]:
concate_string_list_dict_into_simple_string(movie_genres)
concate_string_list_dict_into_simple_string(movie_spoken_language)

In [ ]:
#We will concatenate the three text-based reviews together to form a big variable so 
# that we can use tfidf to produce item_feature matrix
movie_str_criteria = dp(movie_genres)
for index in range(len(movie_genres)):
    movie_str_criteria[index] += movie_genres[index] + movie_spoken_language[index]

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
movie_matrix = tfidf.fit_transform(movie_str_criteria)

In [ ]:
numerical_data = movie_data[['runtime', 'popularity', 'vote_average']]

In [ ]:
numerical_data = numerical_data.fillna(0.0)

discoverered that popularity has a string that is called 'Beware of Frost Bites' and another has 'hello'

In [ ]:
numerical_data.loc[numerical_data['popularity'] == 'Beware Of Frost Bites' , 'popularity'] = float(0)
numerical_data.loc[numerical_data['popularity'] == 'hello' , 'popularity'] = float(0)

In [ ]:
# We use .astype(float) to convert data that contains numerical 
# or decimal numbers will be turned into float
numerical_data['popularity'] = numerical_data['popularity'].astype(float)